In [1]:
%pylab inline

from nltk.stem import WordNetLemmatizer, SnowballStemmer
from gensim.parsing.preprocessing import STOPWORDS
from gensim.utils import simple_preprocess
from gensim.models import TfidfModel
from pathlib import Path
from tempfile import gettempdir
from pyLDAvis.gensim import prepare as lda_prepare
import kaggle
import gensim
import nltk
import numpy as np
import pandas as pd
import pyLDAvis
import warnings

pd.options.display.float_format = '{:,.4f}'.format
nltk.download('wordnet')
kaggle.api.authenticate()
pyLDAvis.enable_notebook()
warnings.simplefilter('ignore')

Populating the interactive namespace from numpy and matplotlib


[nltk_data] Downloading package wordnet to /home/anderson/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


# Data 

In [2]:
dest_path = Path(gettempdir()) / 'news-headlines'
data_name = 'therohk/million-headlines'
kaggle.api.dataset_download_files(data_name, path=dest_path, unzip=True)

data = pd.read_csv(dest_path / 'abcnews-date-text.csv')
data.head()

,publish_date,headline_text
0,20030219,aba decides against community broadcasting lic...
1,20030219,act fire witnesses must be aware of defamation
2,20030219,a g calls for infrastructure protection summit
3,20030219,air nz staff in aust strike for pay rise
4,20030219,air nz strike to affect australian travellers


# Preprocess

## Stemmer Example

Stem은 단어에서 몇개의 글자들을 제거합니다. <br>
따라서 의미없는 단어나 오타등으로 이어질 수 있습니다. 

In [3]:
stemmer = SnowballStemmer('english')

original_words = ['caresses', 'flies', 'dies', 'mules', 'denied', 'died', 'agreed', 'owned',
                  'humbled', 'sized', 'meeting', 'stating', 'siezing', 'itemization', 'sensational',
                  'traditional', 'reference', 'colonizer', 'plotted']
singles = [stemmer.stem(plural) for plural in original_words]
pd.DataFrame(data={'stemmed': singles}, index=original_words).T

,caresses,flies,dies,mules,denied,died,agreed,owned,humbled,sized,meeting,stating,siezing,itemization,sensational,traditional,reference,colonizer,plotted
stemmed,caress,fli,die,mule,deni,die,agre,own,humbl,size,meet,state,siez,item,sensat,tradit,refer,colon,plot


## Lemmatize Example

Context를 이해하고, 단어를 의미있는 base 형태로 변경하며, 변경된 형태를 Lemma 라고 함. 

In [4]:
WordNetLemmatizer().lemmatize('found', pos='v')

'find'

## Tokenization

In [5]:
stemmer = SnowballStemmer('english')
lemmatizer = WordNetLemmatizer()

def lemmatize_stemming(text):
    return stemmer.stem(lemmatizer.lemmatize(text, pos='v'))

def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

doc_sample = data[data.index == 4310]['headline_text'].values[0]

print('original  :', doc_sample)
print('preprocess:', preprocess(doc_sample))

original  : ratepayers group wants compulsory local govt voting
preprocess: ['ratepay', 'group', 'want', 'compulsori', 'local', 'govt', 'vote']


In [6]:
docs = data['headline_text'].apply(preprocess)
docs.head()

0     [decid, communiti, broadcast, licenc]
1                        [wit, awar, defam]
2    [call, infrastructur, protect, summit]
3               [staff, aust, strike, rise]
4      [strike, affect, australian, travel]
Name: headline_text, dtype: object

## Bag of Words

In [7]:
def show_dict(data, n=10):
    i = iter(data.iteritems())

    for _ in range(10):
        k, v = next(i)
        print(k, v)

# To Dictionary
dictionary = gensim.corpora.Dictionary(docs)
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

show_dict(dictionary)

0 broadcast
1 communiti
2 decid
3 licenc
4 awar
5 defam
6 wit
7 call
8 infrastructur
9 protect


In [8]:
# Bag of Words
# [[(index, 횟수), ....], ...]
bow_corpus = [dictionary.doc2bow(doc) for doc in docs]
bow_corpus[0]

[(0, 1), (1, 1), (2, 1), (3, 1)]

In [9]:
def display_bow(bow):
    for idx, cnt in bow:
        word = dictionary[idx]
        print(f'{idx:<6}: {word:15} | count:{cnt}')
    
display_bow(bow_corpus[502])

34    : council         | count:1
1216  : approv          | count:1
1217  : farm            | count:1
1218  : poultri         | count:1


## TF-IDF

In [10]:
tfidf = TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]

print('corpus_tfidf size:', len(corpus_tfidf))oups(shuffle=True, random_state=1,
                             remove=('headers', 'footers', 'quotes'),
                             retu
corpus_tfidf[0]

corpus_tfidf size: 1186018


[(0, 0.5850076620505259),
 (1, 0.38947256567331934),
 (2, 0.4997099083387053),
 (3, 0.5063271308533074)]

# LDA with Bag of Words

## Model

In [11]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary, passes=2, workers=2)
print('num terms :', lda_model.num_terms)
print('num topics:', lda_model.num_topics)

num terms : 14939
num topics: 10


## Feature Importance

In [12]:
def show_lds_feature_importance(lda_model, dictionary, k=10):
    topic_matrix = lda_model.get_topics()

    # argsort with descending order
    feat_impt_indices = (-topic_matrix).argsort(axis=1)[:, :k]
    feat_impt_words = [[dictionary[w] for w in sentence]
                       for sentence in feat_impt_indices]
    feat_impt = np.vstack([topic_matrix[i, idx]
                           for i, idx in enumerate(feat_impt_indices)])

    n_category = len(feat_impt)
    df = pd.concat([pd.DataFrame([feat_impt_words[i], feat_impt[i]], index=['word', 'impt']) for i in range(n_category)],
                   keys=[i for i in range(n_category)]).T

    vis_df = df.style.format('{:.4}') \
               .applymap(lambda x: 'background-color: #e6fcfc', subset=df.columns.get_loc_level('word', level=1)[0]) \
               .bar(subset=df.columns.get_loc_level('impt', level=1)[0], axis=1, width=100, color='#ee1f5f') \
               .set_properties(subset=df.columns.get_loc_level('impt', level=1)[0], width='0px') \
               .set_properties(subset=df.columns.get_loc_level('word', level=1)[0], width='0px')
    display(vis_df)


show_lds_feature_importance(lda_model, dictionary)

In [13]:
lda_vis = lda_prepare(lda_model, bow_corpus, dictionary)
pyLDAvis.display(lda_vis)

# LDA with TF-IDF

## Model

In [14]:
oups(shuffle=True, random_state=1,
                             remove=('headers', 'footers', 'quotes'),
                             retulda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=10, id2word=dictionary, passes=2, workers=4)

## Feature Importance

In [15]:
show_lds_feature_importance(lda_model_tfidf, dictionary)

In [16]:
lda_vis_tfidf = lda_prepare(lda_model_tfidf, bow_corpus, dictionary)
pyLDAvis.display(lda_vis_tfidf)

# Inference

## Inference from Training Set

In [17]:
probs = lda_model_tfidf.inference(bow_corpus[:2])[0]
y_preds = np.argmax(probs, axis=1)
y_scores = probs[arange(len(y_preds)), y_preds]

print('y_preds:', y_preds)
print('y_scores:', y_scores)

y_preds: [6 7]
y_scores: [1.738477  1.0997581]


## Inference from Unseen Document

In [18]:
def inference(lda_model, docs):
    bows = [dictionary.doc2bow(preprocess(doc)) for doc in docs]
    probs = lda_model_tfidf.inference(bows)[0]
    y_preds = np.argmax(probs, axis=1)
    y_scores = probs[arange(len(y_preds)), y_preds]
    y_features = [lda_model.print_topic(y, 5) for y in y_preds]

    print('y_preds:', y_preds)
    print('y_scores:', y_scores)
    
    df = pd.DataFrame({'topic': y_preds, 'score': y_scores, 'feature': y_features})
    vis_df = df.style.set_properties(subset=['feature'], width='500px')
    display(vis_df)

unseen_document = ["Son Heung-min's new Tottenham deal close as Toby Alderweireld reacts to Jose Mourinho decision", 
                   'Heung-min Son thinks £23m Tottenham player is the best at one thing', 
                   'Superstar Son Heung-min is determined to win international ‘A’ match', 
                   'Spurs ace Son Heung-min says he misses £18m man ‘so much’, Mauricio Pochettino sold him']
    
inference(lda_model_tfidf, unseen_document)

y_preds: [4 8 6 8]
y_scores: [2.947288  2.7669137 1.8746536 4.099104 ]


,topic,score,feature
0,4,2.947288,"0.017*""elect"" + 0.013*""drum"" + 0.008*""tuesday"" + 0.008*""labor"" + 0.008*""liber"""
1,8,2.766914,"0.021*""news"" + 0.016*""interview"" + 0.013*""rural"" + 0.009*""live"" + 0.008*""christma"""
2,6,1.874654,"0.015*""crash"" + 0.011*""die"" + 0.010*""hobart"" + 0.007*""fiji"" + 0.007*""violenc"""
3,8,4.099104,"0.021*""news"" + 0.016*""interview"" + 0.013*""rural"" + 0.009*""live"" + 0.008*""christma"""
